In [250]:
import pandas as pd
import numpy as np
import yfinance as yf
import re
from fuzzywuzzy import fuzz
from countrygroups import EUROPEAN_UNION

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [192]:
zones = ["", "China","emerg","Europe","Global","India","Japan","Rest"]

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_spread_classifier(size=5e9, financial=False):
    path = "data_damodaran/capital_structure/spread_classifiers/ratings.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Start here Ratings sheet"]
    moody = process_df(x.iloc[16:32,:4])
    moody_financefirms = process_df(x.iloc[16:,5:9])
    moody_smallfirms = process_df(x.iloc[35:,:4])
    
    if financial == True:
        return moody_financefirms
    else:
        if size >= 5e9:
            return moody
        else: 
            return moody_smallfirms
        
        
def get_10y_tbills():
    path = "data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["T. Bond yield & return"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x

def get_moody_spread(x, size=5e9, financial=False):
    for i in range(len(get_spread_classifier(size,financial=financial))):
        if x <= get_spread_classifier(size,financial=financial).iloc[i,1]:
            return get_spread_classifier(size,financial=financial).iloc[i,-1]
        else:
            continue
    return get_spread_classifier(size,financial=financial).iloc[-1,-1]

industries = list(get_capex().index)

def get_industry(ticket):
    ls = []
    ind = yf.Ticker(ticket).info.get("industry")
    for i in industries:
        ls.append([fuzz.ratio(ind, i),i])
    return pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]

def get_risk_premium():
    path = "data_damodaran/discount_rate_estimation/risk_premium/ctrypremJuly23.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["ERPs by country"]
    x = x.iloc[5:163,:9]
    x = process_df(x)
    x = x.set_index("Country")
    return x


def zones(x):
    if x == "United States":
        return ""
    if x == "China":
        return "China"
    if x == "Canada":
        return "emerg"
    if x == "Australia":
        return "emerg"
    if x == "New Zeland":
        return "emerg"
    if x in EUROPEAN_UNION.names:
        return "Europe"
    if x == "India":
        return "India"
    if x == "Japan":
        return "Japan"
    else:
        return "Global"

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Africa,Moody's rating,Rating-based Default Spread,Total Equity Risk Premium,Country Risk Premium,"Sovereign CDS, net of US",Total Equity Risk Premium2,Country Risk Premium3
Country,,,,,,,,
Abu Dhabi,Middle East,Aa2,0.005281,0.05749,0.00749,0.0027,0.053829,0.003829
Albania,Eastern Europe & Russia,B1,0.048154,0.118298,0.068298,NaN,NaN,NaN
Andorra (Principality of),Western Europe,Baa2,0.020368,0.078888,0.028888,NaN,NaN,NaN
Angola,Africa,B3,0.069528,0.148613,0.098613,0.1004,0.192399,0.142399
Argentina,Central and South America,Ca,0.128369,0.232068,0.182068,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Uruguay,Central and South America,Baa2,0.020368,0.078888,0.028888,0.0052,0.057375,0.007375
Uzbekistan,Eastern Europe & Russia,Ba3,0.038473,0.104567,0.054567,NaN,NaN,NaN
Venezuela,Central and South America,C,0.175,0.298205,0.248205,0.1114,0.208,0.158


In [223]:
x = yf.Ticker("MSFT")
x.info.get("industry")

sector = "Software (System & Application)"

### Capital Structure

In [224]:
E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]
print("Debt:",D,"Equity:",E)

Debt: 205753000000.0 Equity: 206223000000.0


### free-risk rate

In [225]:
rf = get_10y_tbills().iloc[-1,1]
rf

0.0388

### Rd Damodaran Estimation

In [226]:
rd = get_wacc().loc[sector,"Cost of Debt"]

In [229]:
interest = x.incomestmt.loc["Interest Expense",:]
ebit = x.incomestmt.loc["EBIT",:]
coverage_ratio = ebit / interest
spread = get_moody_spread(coverage_ratio.iloc[-1])
rd = rf + spread

### Re Damodaran Estimation

In [230]:
re = get_wacc().loc[sector,"Cost of Equity"]

## re estimated

In [233]:
tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]
beta_u = get_unleveraged_betas().loc[sector,"Unlevered beta"]
beta = beta_u * (1 + (1-tax_rate)*D/E)
prm = get_risk_premium().loc[x.info["country"],"Total Equity Risk Premium"]
re = rf + beta * prm

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0.16379378377367954

In [239]:
get_unleveraged_betas()
beta

2.4998756754735902

## WACC

In [237]:
def wacc(rd,re,t,E,D):
    wacc = rd*(1-t)*(D/(E+D)) + re*(E/(D+E))
    return wacc



In [238]:
wacc(rd,re,tax_rate,E,D)

0.10096140142386192

In [241]:
def wacc(ticket,sector):
    E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
    D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]
    
    rf = get_10y_tbills().iloc[-1,1]
    rd = get_wacc().loc[sector,"Cost of Debt"]
    
    interest = x.incomestmt.loc["Interest Expense",:]
    ebit = x.incomestmt.loc["EBIT",:]
    coverage_ratio = ebit / interest
    spread = get_moody_spread(coverage_ratio.iloc[-1])
    rd = rf + spread
    
    tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]
    beta_u = get_unleveraged_betas().loc[sector,"Unlevered beta"]
    beta = beta_u * (1 + (1-tax_rate)*D/E)
    prm = get_risk_premium().loc[x.info["country"],"Total Equity Risk Premium"]
    re = rf + beta * prm
    
    wacc = rd*(1-tax_rate)*(D/(E+D)) + re*(E/(D+E))
    return wacc

wacc("MSFT", "Software (System & Application)")


/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0.10096140142386192

In [249]:
def wacc(ticket,sector):
    E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
    D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]
    
    rf = get_10y_tbills().iloc[-1,1]
    rd = get_wacc().loc[sector,"Cost of Debt"]
    
    interest = x.incomestmt.loc["Interest Expense",:]
    ebit = x.incomestmt.loc["EBIT",:]
    coverage_ratio = ebit / interest
    spread = get_moody_spread(coverage_ratio.iloc[-1])
    rd = rf + spread
    
    tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]
    beta = x.info.get("beta")
    prm = get_risk_premium().loc[x.info["country"],"Total Equity Risk Premium"]
    re = rf + beta * prm
    
    wacc = rd*(1-tax_rate)*(D/(E+D)) + re*(E/(D+E))
    return wacc

wacc("MSFT", "Software (System & Application)")

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0.06054345154047184

0.885